# Vergleichen verschiedener Embeddings 

In diesem Notebook werden die verschiedenen Embedding Algorithmen demonstriert.

## Imports

In [2]:
import sys

sys.path.append("..")

import json

import joblib
import pandas as pd
from db_connect import db_get_df, db_save_df
from embedding_creation.embedding_creator_TF_IDF import calc_all_tf_idf
from sklearn.metrics.pairwise import cosine_similarity
from tqdm import tqdm
from transformers import AutoModel, AutoTokenizer

/Users/br/Projects/Bachelorarbeit/.venv/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## TF-IDF für alle Daten

### imports

In [ ]:
import os
import sys

import numpy as np
from dotenv import load_dotenv
from scipy import sparse
from sklearn.feature_extraction.text import TfidfVectorizer
import pickle
import joblib
import spacy

sys.path.append("..")

from db_connect import db_get_df, db_save_df, load_pkl, save_pkl, save_npz, load_npz
from segment_ranking.rank_segments import get_most_similar_segments


load_dotenv()
DATA_PATH = os.getenv("DATA_PATH")

### Einfache TF-IDF bilden

In [ ]:
df = db_get_df("transcript_sentences")
tfidf_vectorizer = TfidfVectorizer()
tfidf_matrix = tfidf_vectorizer.fit_transform(df['sentence'])

In [ ]:
print(len(tfidf_vectorizer.get_vocab()))

In [ ]:
save_pkl(tfidf_vectorizer,"tfidf_vectorizer_compound_split_87k.pkl")
save_npz(tfidf_matrix, "tf_idf_matrix_compound_split_87k.npz")

In [ ]:
df = db_get_df(table="sentences_lemmatized")
tfidf_vectorizer = TfidfVectorizer()
tfidf_matrix = tfidf_vectorizer.fit_transform(df['sentence'])

### Lemmatisieren der Daten

In [ ]:
tf_idf_matrix = sparse.load_npz("tf_idf_matrix.npz")
tfidf_vectorizer = joblib.load('tfidf_vectorizer.pkl')
feature_names = tfidf_vectorizer.get_feature_names_out()
idf_values = tfidf_vectorizer.idf_
idf_dict = dict(zip(feature_names, idf_values))
nlp = spacy.load("de_core_news_md")

def lemmatize_german_sentence(input_sentence, nlp):
    doc = nlp(input_sentence)
    lemmatized_words = []
    for token in doc:
        lemma = token.lemma_
        if lemma:
            lemmatized_words.append(lemma)
        else: 
            lemmatized_words.append(token)
    return lemmatized_words

In [ ]:
df = db_get_df("sentences_lemmatized")

In [ ]:
sentence = "Wer ist frau meier"
sentence = lemmatize_german_sentence(sentence, nlp)

encoded_words = [(idf_dict[word.lower()], word) for word in sentence if word.lower() in idf_dict]
print(encoded_words)

### Abspeichern

In [ ]:
save_npz('tf_idf_matrix200k.npz', tfidf_matrix)

### Laden der Modelle

In [ ]:
tf_idf_matrix = load_npz('tf_idf_matrix_230k.npz')
tfidf_vectorizer = load_pkl('tfidf_vectorizer_230k.pkl')

In [ ]:
len(tf_idf_matrix)

### Optional speichern des Vocabulars

In [ ]:
sorted_dict = dict(sorted(tfidf_vectorizer.get_vocab().items()))
output_file = 'vocabulary.txt'
with open(output_file, 'w') as file:
    for key in sorted_dict.keys():
        file.write(key + '\n')


### Demonstration

In [1]:
calc_all_tf_idf()

NameError: name 'calc_all_tf_idf' is not defined

In [ ]:
from scripts.Embedding_creation.embedding_creator_TF_IDF import calc_all_tf_idf, calculate_distances_batchwise


df_tfidf =  calculate_distances_batchwise("Geschichte von Deutschland")

### Lemmatisierung

## Sentence Transformer

SBert
synchron - asynchron

Demonstration

In [4]:
import sys
sys.path.append('..')

from db_connect import db_get_df, db_save_df, save_pkl, load_pkl
from sentence_transformers import SentenceTransformer
from embedding_creator_MINI_L6 import MINI_LM_embed
from segment_ranking.rank_segments import get_most_similar_segments

In [5]:
embeddings = load_pkl("MINI_L6_embeddings.pkl")

In [6]:
embeddings[0:1000].shape

(1000, 384)

In [7]:
get_most_similar_segments("MINILM","Oktoberfest München", 4, 4)

No embedding method for model type MINILM found
No model for model type MINILM found


AttributeError: 'NoneType' object has no attribute 'shape'

In [ ]:
model = SentenceTransformer('all-MiniLM-L6-v2')
sentences = [
    'This framework generates embeddings for each input sentence',
    'Sentences are passed as a list of string.', 
    'The quick brown fox jumps over the lazy dog.']


embeddings = model.encode(sentences)
save_pkl(sentences, embeddings, "test.pkl")

In [ ]:
embeddings = load_pkl("MINI_L6_embeddings.pkl")

In [ ]:
embeddings[0]

In [ ]:
df = db_get_df(table="transcript_sentences")

In [ ]:
df = calculate_distances("Oktoberfest in München", df)

In [ ]:
import sys
sys.path.append('..')

from db_connect import db_get_df, db_save_df, save_pkl, load_pkl
from sentence_transformers import SentenceTransformer
from embedding_creator_MINI_L6 import all_document_embeddings_batchwise_MINI_LM
import pickle

In [ ]:
model = SentenceTransformer('all-MiniLM-L6-v2')
sentences = ['This framework generates embeddings for each input sentence',
    'Sentences are passed as a list of string.', 
    'The quick brown fox jumps over the lazy dog.']


embeddings = model.encode(sentences)
save_pkl(sentences, embeddings, "test.pkl")

In [ ]:
sentences, embeddings = load_pkl("MINI_L6_embeddings.pkl")

In [ ]:
len(embeddings)

In [ ]:
df = db_get_df(table="transcript_sentences")

In [ ]:
embeddings = all_document_embeddings_batchwise_MINI_LM(df["sentence"])
save_pkl(sentences, embeddings, "MINI_LM.pkl")

## Sentece Transformer + TF-IDF

In [ ]:
import sys
sys.path.append('..')

from db_connect import db_get_df, db_save_df, save_pkl, load_pkl, save_npz, load_npz
import scipy.sparse as sparse
import numpy as np

In [ ]:
mini_lm_matrix = load_pkl("MINI_L6_embeddings.pkl")
tf_idf_matrix = load_npz("tf_idf_matrix_230k.npz")

In [ ]:
print(mini_lm_matrix.shape)
print(tf_idf_matrix.shape)

In [ ]:
if not isinstance(mini_lm_matrix, np.ndarray):
    mini_lm_embeddings = np.array(mini_lm_matrix)

mini_lm_sparse_matrix = sparse.csr_matrix(mini_lm_matrix)

combined_matrix = sparse.hstack([tf_idf_matrix, mini_lm_sparse_matrix], format="csr")

In [ ]:
save_npz(combined_matrix, "tf_idf_mini_lm_matrix.npz")

## OpenAI Embeddings

In [2]:
import sys
sys.path.append('..')
from db_connect import db_get_df, db_save_df, save_pkl, load_pkl, save_npz, load_npz
from embeddings_openai import get_embedding_openai
from tqdm import tqdm
import tiktoken
import pandas as pd


MODEL_ID = "text-embedding-3-small"

In [3]:
df = db_get_df("transcript_sentences")

In [35]:
len(df)

433562

In [4]:
encoding = tiktoken.encoding_for_model(MODEL_ID)

In [ ]:
encoding.encode("hey, ich komme heute nicht zur party")

In [ ]:
encoding.decode([36661, 11, 10864, 11129, 2727, 49714, 8969, 17761, 4717])

In [5]:
tokens = [encoding.encode(sentence) for sentence in tqdm(df["sentence"])]

100%|██████████| 433562/433562 [00:24<00:00, 17625.15it/s]


In [6]:
df["token"] = tokens

In [7]:
df['token_count'] = df["token"].apply(len)

In [1]:

# 1424 iterationen
def batch_sentences(df, max_tokens=8191):

    start_idx = 0
    current_tokens = 0
    for i, row in df.iterrows():
        if current_tokens + row['token_count'] > max_tokens:
            # Yield the current batch of sentences
            yield df.iloc[start_idx:i]["token"]
            start_idx = i
            current_tokens = row['token_count']
        else:
            current_tokens += row['token_count']

    if start_idx < len(df):
        yield df.iloc[start_idx:]["token"]



In [ ]:
batch = next(batch_sentences(df))
solo_embed = get_embedding_openai(batch.to_list(), MODEL_ID)
print(solo_embed)

In [ ]:
print(len(solo_embed))
print(solo_embed[4])

In [ ]:
embeddings = load_pkl("embeddings_OPENAI_252769.pkl")

In [9]:
embeddings = []

In [28]:
df_temp = df.iloc[252769:].reset_index(drop=True)

In [29]:
batch = next(batch_sentences(df_temp))

In [30]:
batch

0      [33717, 289, 8154, 55202, 305, 42303, 9267, 89...
1      [17812, 6127, 10709, 1941, 331, 2357, 372, 408...
2      [17812, 9072, 386, 16317, 14104, 2815, 43886, ...
3      [17812, 16095, 2815, 31331, 5086, 268, 2563, 2...
4      [5001, 86, 20578, 48108, 11168, 2815, 44193, 2...
                             ...                        
305    [18674, 469, 485, 12928, 89285, 11, 5568, 2761...
306    [41, 799, 2807, 747, 531, 96138, 54265, 473, 3...
307    [50, 361, 1344, 2448, 14244, 295, 9267, 304, 6...
308    [6219, 52392, 23935, 13045, 62734, 11586, 301,...
309            [77968, 18955, 380, 11, 294, 662, 71, 13]
Name: token, Length: 310, dtype: object

In [31]:
embedding_batch = get_embedding_openai(batch.to_list(), MODEL_ID)

In [15]:
df.iloc[4]

filename                   baruch-de-spinoza-die-suche-nach-der-wahrheit.mp3
sentence                   Mit der Zustimmung des Heiligen Gottes und die...
start                                                                  49.82
end                                                                    72.56
sentence_lemmatized        Mit der Zustimmung des Heiligen Gottes und die...
sentence_compound_split    Mit der Zustimmung des Gottes und dieser verkü...
segment_id                                                                 4
token                      [55470, 2761, 94816, 12828, 2234, 951, 1283, 3...
token_count                                                               67
Name: 4, dtype: object

In [32]:
for batch in tqdm(batch_sentences(df_temp)):
    embedding_batch = get_embedding_openai(batch.to_list(), MODEL_ID)
    for embedding in embedding_batch:
        embeddings.append(embedding.embedding)

591it [42:17,  4.29s/it]


In [33]:
len(embeddings)

180793

In [ ]:
df = pd.DataFrame({"embeddings":embeddings})

In [34]:
save_pkl(embeddings, "embeddings_OPENAI_180793.pkl")

/Users/br/Projects/Bachelorarbeit/data/matrices/embeddings_OPENAI_180793.pkl


In [ ]:
embeddings

In [ ]:
df["embedding"] = embeddings

In [ ]:
def search_reviews(df, product_description, n=3, pprint=True):
   embedding = get_embedding(product_description, model='text-embedding-3-small')
   df['similarities'] = df.ada_embedding.apply(lambda x: cosine_similarity(x, embedding))
   res = df.sort_values('similarities', ascending=False).head(n)
   return res

## LLAMA 2 Embeddings

### Laden der Modelle

In [ ]:
model = AutoModel.from_pretrained('mesolitica/llama2-embedding-1b-8k', trust_remote_code = True)
tokenizer = AutoTokenizer.from_pretrained('mesolitica/llama2-embedding-1b-8k')

In [ ]:
df = db_get_df("transcript_sentences")

### Tokenisierung der Sätze

In [ ]:
input_ids = tokenizer(
    df["sentence"].to_list(), 
    return_tensors = 'pt',
    padding = True
)

In [ ]:
v = model.encode(input_ids).detach().numpy()
v.shape

### Speichern der Embeddings

In [ ]:
df["embedding_json"] = [json.dumps(model.encode(chunk_text).detach().numpy()) for chunk_text in tqdm(input_ids)]


In [ ]:
df = db_get_df()
len(df)

In [ ]:
table_id = list("bcdyefghij")
all_df = db_get_df("transcript_segments_llama_2_a")
for id in table_id:
    df_temp = db_get_df(f"transcript_segments_llama_2_{id}")
    # print(len(df_temp))
    print(df_temp.head(1).iloc[0, 0])
    all_df = pd.concat([all_df, df_temp])


In [ ]:
db_save_df(all_df, "transcript_segments_llama_2_all")

In [ ]:
all_df = db_get_df("transcript_segments_llama_2_all")

In [ ]:
df = db_get_df()

In [ ]:
json_strings = [json.dumps(row.tolist()) for index, row in all_df.iterrows()]
df["embedding_json"] = json_strings

In [ ]:
df.iloc[3]

In [ ]:
db_save_df(df, "transcript_segments_llama_2")

## Voyage Embeddings

In [17]:
from transformers import AutoTokenizer
from tqdm import tqdm
from db_connect import save_pkl, db_get_df

In [65]:

import os

import voyageai
from dotenv import load_dotenv
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("voyageai/voyage")

load_dotenv()
API_KEY = os.getenv("VOYAGE_API_KEY")

vo = voyageai.Client()


def get_embedding_voyage(input, model="voyage-lite-02-instruct"):
    if isinstance(input, str):
        input = [input.replace("\n", " ")]

    # input = json.dumps(input)
    # print(input)
    result = vo.embed(input, model=model, input_type="document")
    return result.embeddings


In [79]:
def batch_sentences(df, max_tokens=8191, max_sentences=128):
    start_idx = 0
    current_tokens = 0
    sentence_count = 0

    for i, row in enumerate(df.itertuples()):  # Using itertuples() for efficiency
        sentence_tokens = row.token_count  # Assuming 'token_count' column exists
        if current_tokens + sentence_tokens > max_tokens or sentence_count == max_sentences:
            # Yield the current batch of sentences and reset the counters
            yield df.iloc[start_idx:i]["sentence"]
            start_idx = i
            current_tokens = sentence_tokens
            sentence_count = 1  # Reset count and include current sentence in the new batch
        else:
            current_tokens += sentence_tokens
            sentence_count += 1

    # Yield the last batch if there are remaining sentences
    if start_idx < len(df):
        yield df.iloc[start_idx:]["sentence"]


In [5]:
df = db_get_df("transcript_sentences")

In [24]:
tokenizer = AutoTokenizer.from_pretrained('voyageai/voyage')
tokens = [tokenizer.encode(sentence) for sentence in tqdm(df["sentence"])]

100%|██████████| 433562/433562 [01:19<00:00, 5473.22it/s]


In [26]:
df["token"] = tokens

In [27]:
df['token_count'] = df["token"].apply(len)

In [32]:
df['token_count']

23

In [45]:
embeddings = []

In [87]:
for batch in tqdm(batch_sentences(df.iloc[194851:], 3000)):
    embedding_batch = get_embedding_voyage(batch.to_list())
    # print(len(embedding_batch))
    for embedding in embedding_batch:
        embeddings.append(embedding)
save_pkl(embeddings, "embeddings_voyage")

0it [00:00, ?it/s]

2282it [1:10:37,  1.86s/it]


/Users/br/Projects/Bachelorarbeit/data/matrices/embeddings_voyage


In [88]:
len(embeddings)

433562

In [86]:
len(df.iloc[194851:])

238711

In [82]:
save_pkl(embeddings, "voyage_embeds_194851")

/Users/br/Projects/Bachelorarbeit/data/matrices/voyage_embeds_194851


In [ ]:
batch_sentences(df)

In [49]:
len(embeddings[0])

1024

In [54]:
gen = batch_sentences(df, 3000)
print(next(gen).to_list())
print(next(gen).to_list())
print(next(gen).to_list())



['Verfolgt und ausgegrenzt.', 'Der niederländische Philosoph Baruch de Spinoza ist schon im 17. Jahrhundert für die demokratische Verfassung eines Staates eingetreten.', 'Die Existenz eines jüdisch -christlichen Schöpfergottes hat er verneint.', 'Nach dem Beschluss der Engel und dem Zeugnis der Heiligen bannen, verstoßen, verwünschen und verfluchen wir Baruch de Spinoza.', 'Mit der Zustimmung des Heiligen Gottes und dieser verkündet die jüdische Gemeinde in Amsterdam mit diesen drastischen Worten, dass sie ihr Gemeindemitglied, den 23 -jährigen Baruch de Spinoza, ab sofort aus ihrer Mitte ausschließt.', 'Mitte des 17. Jahrhunderts konnte ein solcher Bann das soziale Leben eines Menschen vernichten.', 'Zumindest aber machte er die derart ausgeschlossenen zu gesellschaftlichen Außenseitern, die häufig verleumdet oder sogar verfolgt wurden.', 'Genau das war das Leben, das jetzt vor dem jungen Baruch de Spinoza lag.', 'Am 24. November 1632 wurde Baruch de Spinoza als zweiter Sohn jüdischer

In [71]:
input_list = next(gen).to_list()

In [72]:
len(input_list)

19

In [73]:
embedding_batch = get_embedding_voyage(input_list)

In [77]:
len(embedding_batch[2])


1024

In [55]:
for batch in tqdm(batch_sentences(df, 3000)):
    if len(batch) > 127:
        print(len(batch))

40it [00:00, 87.17it/s]

134
155
133


73it [00:00, 90.19it/s]

138
133
129


126it [00:01, 119.54it/s]

...

134
131
136
134
164
138


4090it [00:34, 127.84it/s]

128
150
135


4125it [00:34, 118.50it/s]

132
132
152
135
129
